In [ ]:
import pandas as pd
from pathlib import Path

source_folder = Path("atp_transformed/2000-2024 with covid.csv")

In [2]:
df = pd.read_csv(source_folder) 

C:\Users\Golbe\AppData\Local\Temp\ipykernel_27812\1679286324.py:1: DtypeWarning: Columns (5,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(source_folder)


In [3]:
# make everything date time
df["tourney_date"] = pd.to_datetime(df["tourney_date"])

# make a column with the years
df["tourney_year"] = df["tourney_date"].dt.year

# get the career length
df_years = (
    df.groupby("player_id")["tourney_year"]
      .agg(first_season="min", last_season="max")
)

# create the years active of the player
df_years["years_active"] = df_years["last_season"] - df_years["first_season"] + 1

In [ ]:

# get the static data for all players
df_statics = df[["player_id","player_name","player_height", "player_country"]]
df_statics = df_statics.drop_duplicates("player_id")

In [5]:
# create the total values for all players
df_totals = (
    df.groupby("player_id")[['minutes','player_rank_points', 'ace', 'double_faults', 
                               'points_on_serve','first_serve_in', '1st_won', '2nd_won', 
                               'service_games','break_points_saved', 'break_points_faced', 'match_outcome']]
            .sum()
            )

In [6]:
# create an average age
df_means = df.groupby("player_id")["player_age"].mean().round()

In [7]:
df_match_count = df.groupby("player_id").agg(matches_played=("match_outcome","count"))

In [ ]:
df_merged = (df_statics
                .merge(df_means, on="player_id",how="left")
                .merge(df_totals, on="player_id",how="left")
                .merge(df_years, on="player_id",how="left")
                .merge(df_match_count, on="player_id",how="left")
             )

df_merged.to_csv(Path("atp_transformed/2000-2024_career_totals.csv"),index=False)